# Forward

In [1]:
import torch

implicit_for = torch.load("implicit_admm_iter_10_bwd_10_forward.pt")
unroll_for = torch.load("unroll_admm_iter_10_bwd_10_forward.pt")

/home/goiv/SafeGBPO/.venv/lib/python3.14/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [3]:
implicit_for.keys()

dict_keys(['yraw', 'd_c', 'sK_final', 'y_final', 'y_scaled'])

In [4]:
unroll_for.keys()

dict_keys(['yraw', 'd_c', 'sK_final', 'y_final', 'y_scaled'])

In [21]:
from sklearn.metrics.pairwise import cosine_distances
import seaborn as sns
import matplotlib.pyplot as plt

for sim in range(implicit_for['yraw'].shape[0]):
    v1 = implicit_for['yraw'][sim, :]
    v2 = unroll_for['yraw'][sim, :]

    print(v1, v2) # Initial actions are different -> backprop is going to be different -> what if we 
    # set them to be the same?

    #distances = cosine_distances(v1, v2)

    # plt.figure(figsize=(3, 3))
    #sns.heatmap(distances, square=True, annot=True, cbar=False, cmap='Blues')
    #plt.title(f"Cosine distance for sim {sim}")
    #plt.show()

tensor([[ 0.0584],
        [-0.0584],
        [ 0.0584],
        [-0.0584],
        [-0.0584],
        [ 0.0584],
        [-0.0584],
        [ 0.0584],
        [ 0.0584],
        [-0.0584],
        [ 0.0000]], dtype=torch.float64) tensor([[-0.0757],
        [-0.0596],
        [-0.0757],
        [-0.0596],
        [ 0.0757],
        [ 0.0596],
        [ 0.0757],
        [ 0.0596],
        [-0.0757],
        [-0.0596],
        [ 0.0663]], dtype=torch.float64)
tensor([[ 0.0534],
        [ 0.2451],
        [ 0.0534],
        [ 0.2451],
        [-0.0534],
        [-0.2451],
        [-0.0534],
        [-0.2451],
        [ 0.0534],
        [ 0.2451],
        [ 0.0000]], dtype=torch.float64) tensor([[-0.1738],
        [ 0.0643],
        [-0.1738],
        [ 0.0643],
        [ 0.1738],
        [-0.0643],
        [ 0.1738],
        [-0.0643],
        [-0.1738],
        [ 0.0643],
        [ 0.0000]], dtype=torch.float64)
tensor([[ 0.1120],
        [ 0.1174],
        [ 0.1120],
        [ 0.1174],


In [ ]:
!export PYTHONPATH=$PYTHONPATH:$(pwd)/src

In [ ]:
import torch
import matplotlib.pyplot as plt

from envs.navigate_seeker import NavigateSeekerEnv
from safeguards.pinet import PinetSafeguard
from safeguards.pinetJAX import PinetJAXSafeguard
import time

torch.set_default_dtype(torch.float64)

/home/goiv/SafeGBPO/.venv/lib/python3.14/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


# Not optimized results

In [2]:
def test_pinet_collision():

    # Initialize environment
    env = NavigateSeekerEnv(
        num_envs=1,
        num_steps=100,
        num_obstacles=1,
        min_radius=2.0,
        max_radius=4.0,
        draw_safe_action_set=True,
        polytopic_approach=False
    )

    # Reset environment
    env.reset()
    #show_image(env)

    i = 0

    # Create an action pointing directly at the obstacle
    while(i<3):
        obstacle_center = env.obstacles[0].center[0]  # pick first env and first obstacle
        action_to_obstacle = obstacle_center - env.state[:,:2]
        # Scale to ensure collision (make it long enough)
        action = action_to_obstacle
        print("Intended Collision Action:", action)
        env.execute_action(action)
        #show_image(env)
        i+=1

    experiment_configuration = [
        {"regularisation_coefficient": 0.1, "n_iter_admm": 100, "n_iter_bwd": 100, "debug": False},
        {"regularisation_coefficient": 0.1, "n_iter_admm": 100, "n_iter_bwd": 100, "fpi": True, "debug": False},
        {"regularisation_coefficient": 0.1, "n_iter_admm": 100, "n_iter_bwd": 100, "bwd_method": "unroll", "debug": False},
    ]

    for config in experiment_configuration:
        print("\nExperiment Configuration:", config)
        action_dup = action.detach().clone().requires_grad_(True)

        # Apply PinetSafeguard to get safe action
        wrapper = PinetSafeguard(env, **config)
        start_time = time.time()
        safe_action = wrapper.safeguard(action_dup)
        end_time = time.time()
        print("Safeguarding pass took {:.6f} seconds".format(end_time - start_time))
        print("Safe Action:", safe_action)

        # Backward pass for safe_action
        loss = safe_action.norm()
        start_time = time.time()
        loss.backward()
        end_time = time.time()
        print("Backward pass took {:.6f} seconds".format(end_time - start_time))
        print("Gradient w.r.t input action:", action_dup.grad)

def show_image(env):
    frames = env.render()
    for i, frame in enumerate(frames):
        # Convert tensor to numpy array
        # frame shape: (C, H, W)
        img_np = frame.permute(1, 2, 0).cpu().numpy()  # -> (H, W, C)
        
        plt.figure(figsize=(5,5))
        plt.imshow(img_np)
        plt.axis('off')
        plt.title(f"Environment {i}")
        plt.show()

test_pinet_collision()

Intended Collision Action: tensor([[4.0311, 3.1183]])
Intended Collision Action: tensor([[3.6020, 2.7930]])
Intended Collision Action: tensor([[3.2474, 2.5339]])

Experiment Configuration: {'regularisation_coefficient': 0.1, 'n_iter_admm': 100, 'n_iter_bwd': 100, 'debug': False}
Safeguarding pass took 0.007756 seconds
Safe Action: tensor([[1.4543, 1.1819]], grad_fn=<_ProjectImplicitFnBackward>)
Backward pass took 0.035191 seconds
Gradient w.r.t input action: tensor([[0.1878, 0.1866]])

Experiment Configuration: {'regularisation_coefficient': 0.1, 'n_iter_admm': 100, 'n_iter_bwd': 100, 'fpi': True, 'debug': False}
Safeguarding pass took 0.007007 seconds
Safe Action: tensor([[1.4543, 1.1819]], grad_fn=<_ProjectImplicitFnBackward>)
Backward pass took 0.033618 seconds
Gradient w.r.t input action: tensor([[-0.0210,  0.0298]])

Experiment Configuration: {'regularisation_coefficient': 0.1, 'n_iter_admm': 100, 'n_iter_bwd': 100, 'bwd_method': 'unroll', 'debug': False}
Safeguarding pass took 0.

# Testing optimization

In [2]:
# Initialize environment
env = NavigateSeekerEnv(
    num_envs=1,
    num_steps=100,
    num_obstacles=1,
    min_radius=2.0,
    max_radius=4.0,
    draw_safe_action_set=True,
    polytopic_approach=False
)

# Reset environment
env.reset()
#show_image(env)

i = 0

# Create an action pointing directly at the obstacle
while(i<3):
    obstacle_center = env.obstacles[0].center[0]  # pick first env and first obstacle
    action_to_obstacle = obstacle_center - env.state[:,:2]
    # Scale to ensure collision (make it long enough)
    action = action_to_obstacle
    print("Intended Collision Action:", action)
    env.execute_action(action)
    #show_image(env)
    i+=1

experiment_configuration = [
    {"regularisation_coefficient": 0.1, "n_iter_admm": 100, "n_iter_bwd": 25},
]


print("\nExperiment Configuration:", experiment_configuration[0])
action_dup = action.detach().clone().requires_grad_(True)

print("Original Action:", action_dup)

# Apply PinetSafeguard to get safe action
wrapper = PinetSafeguard(env, **experiment_configuration[0])
start_time = time.time()

action = action_dup.unsqueeze(2)  # match (B, D, 1)

# ----- Build Ax ≤ b -----
start_time = time.time()
A, b = wrapper.env.compute_A_b()
end_time = time.time()
print("Building Ax ≤ b took {:.6f} seconds".format(end_time - start_time))

# ADMM projection function with implicit backward that returns the final safe action.
start_time = time.time()
y_safe = wrapper._project_with_implicit(
    action=action,  # (B, D, 1)
    A=A,            # (B, m, D)
    b=b            # (B, m)
)  # returns (B, D)
end_time = time.time()
print("ADMM projection took {:.6f} seconds".format(end_time - start_time))

print("Safe Action:", y_safe)

Intended Collision Action: tensor([[-3.1261, -4.7779]])
Intended Collision Action: tensor([[-2.8093, -4.2919]])
Intended Collision Action: tensor([[-2.5580, -3.8842]])

Experiment Configuration: {'regularisation_coefficient': 0.1, 'n_iter_admm': 100, 'n_iter_bwd': 25}
Original Action: tensor([[-2.5580, -3.8842]], requires_grad=True)
Building Ax ≤ b took 0.001133 seconds
ADMM projection took 0.006756 seconds
Safe Action: tensor([[-0.0893, -0.1205]], grad_fn=<_ProjectImplicitFnBackward>)


### Projection takes more than defining the state

In [2]:
# Initialize environment
env = NavigateSeekerEnv(
    num_envs=1,
    num_steps=100,
    num_obstacles=1,
    min_radius=2.0,
    max_radius=4.0,
    draw_safe_action_set=True,
    polytopic_approach=False
)

# Reset environment
env.reset()
#show_image(env)

i = 0

# Create an action pointing directly at the obstacle
while(i<3):
    obstacle_center = env.obstacles[0].center[0]  # pick first env and first obstacle
    action_to_obstacle = obstacle_center - env.state[:,:2]
    # Scale to ensure collision (make it long enough)
    action = action_to_obstacle
    print("Intended Collision Action:", action)
    env.execute_action(action)
    #show_image(env)
    i+=1

experiment_configuration = [
    {"regularisation_coefficient": 0.1, "n_iter_admm": 15, "n_iter_bwd": 10},
]


print("\nExperiment Configuration:", experiment_configuration[0])
action_dup = action.detach().clone().requires_grad_(True)

print("Original Action:", action_dup)

# Apply PinetSafeguard to get safe action
wrapper = PinetSafeguard(env, **experiment_configuration[0])
start_time = time.time()

safe_action = wrapper.safeguard(action_dup)

print("Safe Action:", safe_action)

Intended Collision Action: tensor([[ 0.4256, -5.4905]])
Intended Collision Action: tensor([[ 0.3774, -4.9389]])
Intended Collision Action: tensor([[ 0.3656, -4.4455]])

Experiment Configuration: {'regularisation_coefficient': 0.1, 'n_iter_admm': 15, 'n_iter_bwd': 10}
Original Action: tensor([[ 0.3656, -4.4455]], requires_grad=True)
Building equality constraints took 0.000086 seconds
Ruiz scaling took 0.000425 seconds
Building constraints took 0.000575 seconds
Forward took 0.020412 seconds
Safe Action: tensor([[ 0.2284, -2.0202]], grad_fn=<_ProjectImplicitFnBackward>)


In [11]:
################ Remove unnecessary .clones() ####################3

# Initialize environment
env = NavigateSeekerEnv(
    num_envs=1,
    num_steps=100,
    num_obstacles=1,
    min_radius=2.0,
    max_radius=4.0,
    draw_safe_action_set=True,
    polytopic_approach=False
)

# Reset environment
env.reset()
#show_image(env)

i = 0

# Create an action pointing directly at the obstacle
while(i<3):
    obstacle_center = env.obstacles[0].center[0]  # pick first env and first obstacle
    action_to_obstacle = obstacle_center - env.state[:,:2]
    # Scale to ensure collision (make it long enough)
    action = action_to_obstacle
    print("Intended Collision Action:", action)
    env.execute_action(action)
    #show_image(env)
    i+=1

experiment_configuration = [
    {"regularisation_coefficient": 0.1, "n_iter_admm": 100, "n_iter_bwd": 10, "debug":True},
]


print("\nExperiment Configuration:", experiment_configuration[0])
action_dup = action.detach().clone().requires_grad_(True)

print("Original Action:", action_dup)

# Apply PinetSafeguard to get safe action
wrapper = PinetSafeguard(env, **experiment_configuration[0])
start_time = time.time()

safe_action = wrapper.safeguard(action_dup)

end_time = time.time()
print("Safeguarding pass took {:.6f} seconds".format(end_time - start_time))

print("Safe Action:", safe_action)

# Backward pass for safe_action
loss = safe_action.norm()
start_time = time.time()
loss.backward()
end_time = time.time()
print("Backward pass took {:.6f} seconds".format(end_time - start_time))
print("Gradient w.r.t input action:", action_dup.grad)

Intended Collision Action: tensor([[ 1.6113, -3.7102]])
Intended Collision Action: tensor([[ 1.4855, -3.3594]])
Intended Collision Action: tensor([[ 1.3264, -3.0654]])

Experiment Configuration: {'regularisation_coefficient': 0.1, 'n_iter_admm': 100, 'n_iter_bwd': 10, 'debug': True}
Original Action: tensor([[ 1.3264, -3.0654]], requires_grad=True)
PinetSafeguard: ADMM projection took 0.004321 seconds
Safeguarding pass took 0.005762 seconds
Safe Action: tensor([[0.0136, 0.0039]], grad_fn=<_ProjectImplicitFnBackward>)
Backward pass took 0.006536 seconds
Gradient w.r.t input action: tensor([[-0.0206, -0.0081]])


In [ ]:
############ Newest version #############

# Initialize environment
env = NavigateSeekerEnv(
    num_envs=1,
    num_steps=100,
    num_obstacles=1,
    min_radius=2.0,
    max_radius=4.0,
    draw_safe_action_set=True,
    polytopic_approach=False
)

# Reset environment
env.reset()
#show_image(env)

i = 0

# Create an action pointing directly at the obstacle
while(i<3):
    obstacle_center = env.obstacles[0].center[0]  # pick first env and first obstacle
    action_to_obstacle = obstacle_center - env.state[:,:2]
    # Scale to ensure collision (make it long enough)
    action = action_to_obstacle
    print("Intended Collision Action:", action)
    env.execute_action(action)
    #show_image(env)
    i+=1

experiment_configuration = [
    {"regularisation_coefficient": 0.1, "n_iter_admm": 100, "n_iter_bwd": 5, "debug": True},
]


print("\nExperiment Configuration:", experiment_configuration[0])
action_dup = action.detach().clone().requires_grad_(True)

print("Original Action:", action_dup)

# Apply PinetSafeguard to get safe action
wrapper = PinetSafeguard(env, **experiment_configuration[0])
start_time = time.time()

safe_action = wrapper.safeguard(action_dup)

end_time = time.time()
print("Safeguarding pass took {:.6f} seconds".format(end_time - start_time))

print("Safe Action:", safe_action)

# Backward pass for safe_action
loss = safe_action.norm()
start_time = time.time()
loss.backward()
end_time = time.time()
print("Backward pass took {:.6f} seconds".format(end_time - start_time))
print("Gradient w.r.t input action:", action_dup.grad)

wrapper = PinetJAXSafeguard(env, **experiment_configuration[0])
start_time = time.time()

safe_action = wrapper.safeguard(action_dup)

end_time = time.time()
print("Safeguarding pass took {:.6f} seconds".format(end_time - start_time))

print("Safe Action:", safe_action)

# Backward pass for safe_action
loss = safe_action.norm()
start_time = time.time()
loss.backward()
end_time = time.time()
print("Backward pass took {:.6f} seconds".format(end_time - start_time))
print("Gradient w.r.t input action:", action_dup.grad)

Intended Collision Action: tensor([[ 4.9218, -0.1863]])
Intended Collision Action: tensor([[ 4.4370, -0.1549]])
Intended Collision Action: tensor([[ 3.9551, -0.1780]])

Experiment Configuration: {'regularisation_coefficient': 0.1, 'n_iter_admm': 100, 'n_iter_bwd': 10, 'debug': True}
Original Action: tensor([[ 3.9551, -0.1780]], requires_grad=True)
PinetSafeguard: ADMM projection took 0.004634 seconds
Safeguarding pass took 0.007555 seconds
Safe Action: tensor([[ 0.1424, -0.0105]], grad_fn=<_ProjectImplicitFnBackward>)
Pairwise Distance: 0.004968694377685947
Pairwise Distance: 0.004561800617056725
Pairwise Distance: 0.0041894634249486865
Pairwise Distance: 0.003849175055695889
Pairwise Distance: 0.003538529018747864
Pairwise Distance: 0.003255217991716996
Pairwise Distance: 0.0029970344587605586
Pairwise Distance: 0.0027618733720962078
Pairwise Distance: 0.0025477361506583735
Pairwise Distance: 0.0023527353469659845
Backward pass took 0.006602 seconds
Gradient w.r.t input action: tensor